In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_responses.csv')

In [ ]:
df.head()

In [ ]:
df.rename(columns = {'What country are you from? ': 'nationality', 'Age Range': 'age', 'Gender': 'gender',
                    '1. Choose your top favorite 3-5 cities you have traveled to that are on this list.  - Favorite City #1': 'favorite_city_one',
                    '2. Favorite city #2': 'favorite_city_two', '3. Favorite city #3': 'favorite_city_three',
                    '4. Favorite city #4': 'favorite_city_four', '5. Favorite city #5': 'favorite_city_five',
                    "6. If there's a city you have been and loved that is not on this list, add it below. ": 'extra_favorite',
                    "7. What cities on this list have you been to and not enjoyed?   - Least favorite city #1": 'least_favorite_one',
                    '8. Least favorite city #2': 'least_favorite_two', 
                    "9. If there's a city you have been to and haven't liked that is not on this list, add it below": 'extra_least_favorite',
                    "What price range of restaurant do you eat at when you travel? [Price level: 1 (Fast/Cheap Eats)]": 'food_one',
                    "What price range of restaurant do you eat at when you travel? [Price level: 2 (Casual Dining)]": 'food_two',
                    "What price range of restaurant do you eat at when you travel? [Price level: 3 (Upscale Dining)]": 'food_three',
                    "What price range of restaurant do you eat at when you travel? [Price level: 4 (Fine Dining/High End)]": 'food_four',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Art Gallery]": 'art_gallery',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Library]": 'library',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Museum ]": 'museum',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Aquarium]": 'aquarium',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Amusement Park ]": 'amusement_park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Zoo]": 'zoo',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Movie Theater]": 'movie_theater',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Mall / Souvenir shop ]": 'store',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Park ]": 'park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Natural Feature / Beach]": 'natural_feature',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Place of Worship (Church/Temple)]": 'place_of_worship'}, inplace=True )

In [ ]:
df.columns

In [ ]:
df.fillna('None', inplace=True)

In [ ]:
df.drop(columns=['Timestamp', 'extra_favorite', 'extra_least_favorite'], inplace=True)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
city_df = df.copy()

In [ ]:
city_df.drop(columns=['nationality', 'age', 'gender'], inplace=True)
city_df

In [ ]:
city_df.reset_index(drop=False, inplace=True)
city_df.rename(columns={'index': 'user_id'}, inplace=True)

In [ ]:
dummy = pd.get_dummies(df)
dummy_city = pd.get_dummies(city_df)

In [ ]:
dummy

In [ ]:
dummy_city

In [ ]:
# Not using one hot encoder..
#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
#X_enc = enc.fit_transform(df)
#X_enc

In [ ]:
cosine = cosine_similarity(dummy)
cosine_city = cosine_similarity(dummy_city)

In [ ]:
# city does not have user demographics
cosine_df = pd.DataFrame(cosine)
cosine_df_city = pd.DataFrame(cosine_city)

In [ ]:
cosine_df_city

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
top_10 = find_similar_n(cosine_df,10)
top_10_c= find_similar_n(cosine_df_city,10)

In [ ]:
top_10_c

In [ ]:
top_10

In [ ]:
df.iloc[[149, 130, 110, 105, 0, 54], 3:9]

In [ ]:
df = city_df[['user_id', 'favorite_city_one', 'favorite_city_two', 'favorite_city_three', 'favorite_city_four', 'favorite_city_five']].copy()

In [ ]:
df

In [ ]:
df_backwards = df.rename(columns={'favorite_city_one': 5, 'favorite_city_two': 4, 'favorite_city_three': 3,
                  'favorite_city_four': 2, 'favorite_city_five': 1})
df_backwards

In [ ]:
df.rename(columns={'favorite_city_one': 1, 'favorite_city_two': 2, 'favorite_city_three': 3,
                  'favorite_city_four': 4, 'favorite_city_five': 5}, inplace=True)

In [ ]:
melted = pd.melt(df, id_vars = ['user_id'], value_vars = [1,2,3,4,5])
melted_b = pd.melt(df, id_vars = ['user_id'], value_vars = [5,4,3,2,1])
#price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
#pd.melt(df, id_vars =['Name'], value_vars =['Course']) 


In [ ]:
melted

In [ ]:
melted.rename(columns={'value': 'city', 'variable':'rank'}, inplace=True)
melted = melted[['user_id', 'city', 'rank']]

melted_b.rename(columns={'value': 'city', 'variable':'rank'}, inplace=True)
melted_b = melted[['user_id', 'city', 'rank']]


In [ ]:
melted.sort_values('user_id', inplace=True)
melted.reset_index(drop=True, inplace=True)

melted_b.sort_values('user_id', inplace=True)
melted_b.reset_index(drop=True, inplace=True)

In [ ]:

melted['rank'] = pd.to_numeric(melted['rank'])
melted_b['rank'] = pd.to_numeric(melted['rank'])


In [ ]:
melted_b.drop(melted_b[melted_b.city == 'None'].index, inplace=True)
melted.drop(melted[melted.city == 'None'].index, inplace=True)


In [ ]:
melted

In [ ]:
#suprise_5 = melted.to_csv(r"/Users/tristannisbet/Documents/SM/Dataframe/suprise_5.csv", index=False)
#suprise_b = melted_b.to_csv(r"/Users/tristannisbet/Documents/SM/Dataframe/suprise_b.csv", index=False)


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import KFold
from surprise import accuracy


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
melted.iloc[:, 1] = labelencoder.fit_transform(melted.iloc[:, 1])

In [ ]:
# Change it back to countries
melted.iloc[:, 1] = labelencoder.inverse_transform(melted.iloc[:, 1])

In [ ]:
reader= Reader(rating_scale=(1.0, 3.0))
data= Dataset.load_from_df(melted[['user_id', 'city', 'rank']], reader)


In [ ]:
melted

In [ ]:
kf = KFold(n_splits=3)
algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
mix = melted[['user_id', 'rank', 'city']].copy()

In [ ]:
mix.sort_values('city')

In [ ]:
reader2 = Reader(rating_scale=(1.0, 80.0))
data2 = Dataset.load_from_df(melted[['user_id', 'rank', 'city']], reader)

In [ ]:
kf = KFold(n_splits=3)
algo = SVD()

for trainset, testset in kf.split(data2):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

# Use movielens-100K
#data = Dataset.load_builtin('ml-100k')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])